In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
labels = pd.read_csv("https://raw.githubusercontent.com/Koalananas/pythonfordata/master/datas/features.txt", header=None)
l = labels[0].tolist()
labels = []
#rename features with same name
i = 0
for la in l:
  i += 1
  la = la.replace(" ", "")
  while(la in labels):
    la = la+"x"
  labels.append(la)
  #labels.append(i)

df1x = pd.read_csv("https://raw.githubusercontent.com/Koalananas/pythonfordata/master/datas/Train/X_train.txt"," ", header=None, names=labels)
df1y = pd.read_csv("https://raw.githubusercontent.com/Koalananas/pythonfordata/master/datas/Train/y_train.txt"," ", header=None, names=['y_target'])
df1  = pd.concat([df1x, df1y], axis=1)

df2x = pd.read_csv("https://raw.githubusercontent.com/Koalananas/pythonfordata/master/datas/Test/X_test.txt"," ", header=None, names=labels)
df2y = pd.read_csv("https://raw.githubusercontent.com/Koalananas/pythonfordata/master/datas/Test/y_test.txt"," ", header=None, names=['y_target'])
df2  = pd.concat([df2x, df2y], axis=1)

df = pd.concat([df1, df2])

In [0]:
#test correlation matrix
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [3]:
def svm(trainx, trainy, testx, testy):
  from sklearn import svm
  clf = svm.SVC(gamma=0.001, C=100.)
  clf.fit(trainx, trainy.values.ravel())
  predictions = clf.predict(testx)
  i = 0
  accuracy =0
  for res in predictions:
    if res == testy.loc[i, "y_target"]:
      accuracy +=1
    i+=1
  accuracy = (accuracy/i)
  return [clf, accuracy]

svm(df1x, df1y, df2x, df2y)[1]

0.9535104364326376

In [5]:
def rfc(trainx, trainy, testx, testy):
  from sklearn.ensemble import RandomForestClassifier
  rfc = RandomForestClassifier(n_estimators = 100)
  rfc.fit(trainx,trainy.values.ravel())
  predictions = rfc.predict(testx)
  i = 0
  accuracy =0
  for res in predictions:
    if res == testy.loc[i, "y_target"]:
      accuracy +=1
    i+=1
  accuracy = (accuracy/i)
  return [rfc, accuracy]

rfc(df1x, df1y, df2x, df2y)[1]

0.9152435167615434

In [7]:
def logreg(trainx, trainy, testx, testy):
  from sklearn.linear_model import LogisticRegression
  logres = LogisticRegression(solver='lbfgs', multi_class="ovr", max_iter=1000)
  logres.fit(trainx, trainy.values.ravel())
  predictions = logres.predict(testx)
  i = 0
  accuracy =0
  for res in predictions:
    if res == testy.loc[i, "y_target"]:
      accuracy +=1
    i+=1
  accuracy = (accuracy/i)
  return [logres, accuracy]

logreg(df1x, df1y, df2x, df2y)[1]

0.9468690702087287

In [12]:
modelsvm = svm(df1x, df1y, df2x, df2y)[0]
with open("pythonfordata/models/model", 'wb') as file:
            pickle.dump(modelsvm, file)